In [3]:
from ultralytics import YOLO

model_path = '/home/messyas/ml/jetson/runs/detect/yolov8xtrain/weights/best.pt'
data_yaml_path = '/home/messyas/ml/data/tccdata/yolo_dataset/data.yaml'

try:
    model = YOLO(model_path)
    results = model.val(data=data_yaml_path, split='test', save_json=False)
    map_50_95 = results.box.map
    map_50 = results.box.map50
    try:
        recall = results.results_dict['metrics/recall(B)']
    except KeyError:
        print("A chave 'metrics/recall(B)' não foi encontrada. Verifique as chaves disponíveis:")
        print(results.results_dict.keys())
        recall = None 

    f1_score = None
    if recall is not None:
        try:
            precision = results.results_dict['metrics/precision(B)']
            if (precision + recall) > 0:
                f1_score = 2 * (precision * recall) / (precision + recall)
            else:
                f1_score = 0
        except KeyError:
            precision = None

    print("\n--- Métricas YOLOv8x ---")
    print(f"mAP (50:95): {map_50_95:.4f} ({map_50_95 * 100:.2f} %)")
    print(f"mAP50:       {map_50:.4f} ({map_50 * 100:.2f} %)")

    if recall is not None:
        print(f"Revocação:   {recall:.4f} ({recall * 100:.2f} %)")
    else:
        print("Revocação:   N/A")

    if f1_score is not None:
        print(f"F1-Score:    {f1_score:.4f} ({f1_score * 100:.2f} %)")
    else:
        print("F1-Score:    N/A")

except Exception as e:
    print(f"Ocorreu um erro: {e}")

Ocorreu um erro: [Errno 2] No such file or directory: '/home/messyas/ml/jetson/runs/detect/yolov8xtrain/weights/best.pt'


In [1]:
import cv2
import torch
import os
import random
from ultralytics import YOLO
import numpy as np 

MODEL_PATH = '/home/messyas/ml/jetson/runs/detect/yolov8xtrain/weights/best.pt'
TEST_IMG_DIR = '/home/messyas/ml/data/tccdata/yolo_dataset/images/test/'
DATA_YAML_PATH = '/home/messyas/ml/data/tccdata/yolo_dataset/data.yaml'

OUTPUT_DIR = './output_visualizations_gray/' 
NUM_IMAGES_TO_SAVE = 16
CONF_THRESHOLD = 0.5  
BOX_THICKNESS = 4
FONT_SCALE = 0.5
FONT_THICKNESS = 1

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Carregando modelo de: {MODEL_PATH}")
model = YOLO(MODEL_PATH)
class_names = model.names
print(f"Modelo carregado. Classes: {class_names}")

np.random.seed(42) 
colors = np.random.randint(50, 256, size=(len(class_names), 3), dtype='uint8').tolist()

try:
    all_test_images = [os.path.join(TEST_IMG_DIR, f) for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if not all_test_images:
        print(f"ERRO: Nenhuma imagem encontrada em: {TEST_IMG_DIR}")
        exit()
    if len(all_test_images) < NUM_IMAGES_TO_SAVE:
        print(f"Aviso: Encontradas apenas {len(all_test_images)} imagens, processando todas.")
        selected_images = all_test_images
    else:
        selected_images = random.sample(all_test_images, NUM_IMAGES_TO_SAVE)
    print(f"\nSelecionadas {len(selected_images)} imagens para visualização.")

except FileNotFoundError:
    print(f"ERRO: Diretório de imagens de teste não encontrado: {TEST_IMG_DIR}")
    exit()

for img_path in selected_images:
    print(f"Processando: {os.path.basename(img_path)}...")
    img = cv2.imread(img_path)
    if img is None:
        continue

    results = model.predict(source=img_path, conf=CONF_THRESHOLD, verbose=False)
    result = results[0] 

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_out = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    boxes = result.boxes.xyxy.cpu().numpy()
    scores = result.boxes.conf.cpu().numpy()
    classes = result.boxes.cls.cpu().numpy()

    for i in range(len(boxes)):
        x1, y1, x2, y2 = map(int, boxes[i])
        score = scores[i]
        cls_id = int(classes[i])
        cls_name = class_names[cls_id]
        color = colors[cls_id]


        cv2.rectangle(img_out, (x1, y1), (x2, y2), color, BOX_THICKNESS)

      
        label = f"{cls_name}: {score:.2f}"
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, FONT_THICKNESS)
       
        text_y = y1 - 5 if y1 - 5 > h else y1 + h + 5 
        cv2.rectangle(img_out, (x1, text_y - h), (x1 + w, text_y), color, -1) 
        cv2.putText(img_out, label, (x1, text_y), cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, (0, 0, 0), FONT_THICKNESS) 


    base_name = os.path.basename(img_path)
    save_path = os.path.join(OUTPUT_DIR, f"gray_pred_{base_name}")
    cv2.imwrite(save_path, img_out)
    print(f" imagem salva em: {save_path}")

print(f"\n imagens salvas em '{OUTPUT_DIR}'.")

try:
    val_results = model.val(data=DATA_YAML_PATH, split='test', verbose=False)
    print(f"mAP50-95: {val_results.box.map:.4f}")
    print(f"mAP50:    {val_results.box.map50:.4f}")
    print(f"mAP75:    {val_results.box.map75:.4f}")
except Exception as e:
    print(f"ERRO ao executar a validação: {e}")
    print(f"Verifique o caminho do arquivo YAML: {DATA_YAML_PATH}")

Carregando modelo de: /home/messyas/ml/jetson/runs/detect/yolov8xtrain/weights/best.pt


FileNotFoundError: [Errno 2] No such file or directory: '/home/messyas/ml/jetson/runs/detect/yolov8xtrain/weights/best.pt'

In [2]:
import os
import cv2
from ultralytics import YOLO

MODEL_PATH = '/home/messyas/ml/jetson/runs/detect/yolov8xtrain/weights/best.pt'
MODEL_CONF_THR = 0.05
IMAGE_TO_PROCESS = "/home/messyas/Downloads/imagnesTestReais/imagemdeigarapeaerea.jpeg"
OUTPUT_DIR_JUPYTER = "single_image_output_yolo_jupyter" 

def build_yolo_model(model_path: str) -> YOLO:
    """Carrega o modelo YOLO."""
    if not os.path.exists(model_path):
        print(f"ERRO: Arquivo de pesos YOLO não encontrado: {model_path}")
        return None
    try:
        model = YOLO(model_path)
        print(f"Modelo YOLO carregado de: {model_path}")
        print(f"Classes do modelo: {model.names}")
        return model
    except Exception as e:
        print(f"ERRO ao carregar modelo YOLO: {e}")
        return None

def predict_yolo(model: YOLO, img_path, conf_threshold):
    """Executa a predição com o modelo YOLO em CPU e retorna coordenadas em pixels."""
    results = model.predict(source=img_path, conf=conf_threshold, device='cpu', verbose=False)
    result = results[0]
    boxes = result.boxes.xyxy.cpu().numpy().tolist()
    scores = result.boxes.conf.cpu().numpy().tolist()
    labels = result.boxes.cls.cpu().numpy().tolist()
    return boxes, scores, labels

def draw_enhanced_boxes(img_rgb, filename, predictions, yolo_names, output_dir):
    """Desenha caixas delimitadoras aprimoradas (AZUIS) sobre fundo P&B e salva."""
    img_draw = img_rgb.copy()
    img_gray = cv2.cvtColor(img_draw, cv2.COLOR_RGB2GRAY)
    img_draw = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB)
    img_bgr = cv2.cvtColor(img_draw, cv2.COLOR_RGB2BGR)

    box_color_blue = (255, 100, 0)
    text_color = (255, 255, 255)
    text_bg_color = (0, 0, 0)
    thickness = 4
    font_scale = 0.8
    font_thickness = 2

    for (x1, y1, x2, y2), category_id, score_percentage in predictions:
        cat_name = yolo_names.get(category_id, f'ID {category_id}')
        label = f"{cat_name}: {score_percentage}"
        current_box_color = box_color_blue

        cv2.rectangle(img_bgr, (x1, y1), (x2, y2), current_box_color, thickness)
        (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
        text_y_base = y1 - 10
        if text_y_base - text_height - 10 < 0:
            text_y_base = y2 + text_height + 15
        cv2.rectangle(img_bgr, (x1, text_y_base - text_height - 5), (x1 + text_width, text_y_base + 5), text_bg_color, cv2.FILLED)
        cv2.putText(img_bgr, label, (x1, text_y_base), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness, cv2.LINE_AA)

    output_filename = f"pred_yolo_{os.path.basename(filename)}"
    output_path = os.path.join(output_dir, output_filename)
    cv2.imwrite(output_path, img_bgr)
    print(f"Imagem processada salva em: {output_path}")

def run_prediction_for_jupyter():
    """Função principal adaptada para rodar no Jupyter."""
    img_path = IMAGE_TO_PROCESS
    output_dir = OUTPUT_DIR_JUPYTER
    final_score_threshold = FINAL_THRESHOLD_JUPYTER

    os.makedirs(output_dir, exist_ok=True)

    if not os.path.exists(img_path):
        print(f"ERRO: Imagem de entrada não encontrada: {img_path}")
        return

    print("Carregando modelo YOLOv8...")
    model = build_yolo_model(MODEL_PATH)

    if model is None:
        print("Erro ao carregar o modelo. Abortando.")
        return

    yolo_names = model.names

    print(f"Processando imagem: {img_path}")
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        print(f"ERRO: Não foi possível ler a imagem {img_path}.")
        return

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    print("Executando predição...")
    boxes, scores, labels = predict_yolo(model, img_path, MODEL_CONF_THR)

    print(f"Filtrando resultados (limiar = {final_score_threshold*100:.0f}%)...")
    image_predictions_for_plot = []
    for b, s, l in zip(boxes, scores, labels):
        if s >= final_score_threshold:
            x1, y1, x2, y2 = map(int, b)
            category_id = int(l)
            score_percentage = f"{s:.1%}"
            image_predictions_for_plot.append(((x1, y1, x2, y2), category_id, score_percentage))

    if not image_predictions_for_plot:
        print("Nenhuma detecção encontrada acima do limiar.")
        return

    print(f"Desenhando {len(image_predictions_for_plot)} caixas delimitadoras...")
    draw_enhanced_boxes(img_rgb, img_path, image_predictions_for_plot, yolo_names, output_dir)

    print("Processo concluído.")

run_prediction_for_jupyter()

NameError: name 'FINAL_THRESHOLD_JUPYTER' is not defined